# Лабораторная работа №7

## Постановка задачи

1. Реазизовать Rejection sampling

2. Реазиовать метод Метрополиса-Гастингса

3. Протестировать работу методов на двух функциях плотности распределения

## Источники

* [Метод Парзена-Розенблата](https://yandex.ru/q/machine-learning/12341383425/)


# Подключение зависимостей

In [1]:
import math
import copy
import numpy as np
import random
from matplotlib import pyplot as plt
from enum import Enum
from typing import Tuple, Mapping

## Экспоненциальное распределение

$F_X(x) = \left\{\begin{matrix}
1-e^{-\lambda x}&,\; x \ge 0, \\
0 &,\; x < 0.
\end{matrix}\right.$

In [4]:
class ExponentialDistribution():
  
  @staticmethod
  def generate_value(lmd: float = 0.1) -> float:
    return math.log(1 / (1 - random.uniform(0, 1))) / lmd

  @staticmethod
  def generate_list(lmd: float = 0.1, count: int = 100) -> list:
    return [ExponentialDistribution.generate_value(lmd) for i in range(count)]

## Распределение Вейбулла

$F(x;k,\lambda) = 1- e^{-(x/\lambda)^k}$

In [3]:
class WeibullDistribution():
  
  @staticmethod
  def generate_value(lmd: float = 1.0, k: float = 1.5) -> float:
    return (math.log(1 / (1 - random.uniform(0, 1))) / lmd ** k) ** (1 / k)

  @staticmethod
  def generate_list(lmd: float = 1., k: float = 1.5, count: int = 100) -> list:
    return [WeibullDistribution.generate_value(lmd, k) for i in range(count)]

In [2]:
def normal_dist(x: float, m: float = 0.35 , d: int = 1) -> float:
    return (math.exp((x - m)**2/(-2*d**2))/(d*(2*math.pi)**0.5))

In [ ]:
def rejection_sampling_Weibull(count = 200, m = 0.35, d = 1, c = 1):
  result = []

  while len(result) < count:
    x = np.random.normal(m, 0.5)
    u = np.random.uniform(0, c * normal_dist(x, m, d))
    px = distribution_density_Weibull(x)
    if (isinstance(px, complex) and (px.imag != 0)): continue 
    if(px > u):
      result.append(x)
  return result